<a href="https://colab.research.google.com/github/nisunze/rubavu_rock_excavation/blob/main/site_report_generator_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook takes an xlsx workbook with tables with defined schema, 
makes some necessary calculations, and produce an Excel workbook for
reporting. I will produce a future version in Apache Beam and GCP, and another version using Cloud Dataprep on GCP

In [7]:
#import needed libraries
from pathlib import Path
import pandas as pd
import numpy as np
 
import datetime
yesterday = datetime.date.today() - datetime.timedelta(days=1)
yesterday.strftime("%d/%m/%Y")
 
#directories: input and output
input_workbook = Path(r'/content/drive/MyDrive/Pro/ArtSEC/Rock Excavation in Rubavu/site_operations.xlsx')
output_workbook = input_workbook.parent/'site_report.xlsx'

fuel_price = 1050
# analyzing dataframes data
def make_str_python_val(x):
    return x.lower().replace('.','_').replace(' ', '_')
 
def title_string(x):
    return x.replace('_', ' ').title()
 
def df_cols_title(df):
    '''
    Tile df columns for reporting
    '''
    df.columns = [title_string(l) for l in df.columns.to_list()]
    return df
 
def parse_df_header(df):
    '''
    Parse DataFrame column names into a pythonic format
    Takes a dataframe, and returns a dataframe with formatted headers
    '''
    df.columns = [make_str_python_val(l) for l in df.columns.to_list()]
    return df
 
def dfs_from_xls(input_workbook):
    '''
    Generate dataframes from excel sheets
    '''
    xls = pd.ExcelFile(input_workbook)
    sheet_names = xls.sheet_names
    print(sheet_names)
    xls_df = []
    for sheet_name in sheet_names:
        df = xls.parse(sheet_name=sheet_name)
        xls_df.append(df)
    #print(sheet_names)
    for df in xls_df:
        parse_df_header(df)
    return xls_df #list of dataframes
 
def machine_staats(equipment_log, equipment_db):
    #merge equipment log and equipment database dfs
    df = pd.merge(equipment_log, equipment_db, on='plate') 
    df.sort_values(by='date', inplace=True)
    df['hm_rate_per_minutes'] = df.hummer_price_per_hour / 60
    df['bkt_rate_per_minutes'] = df.bucket_price_per_hour / 60
    df['price_per_hour'] = df['bucket_price_per_hour'].where(df['tool']=='Bucket', df['hummer_price_per_hour'] )
    # df['rate_per_minutes'].round(decimals=2)
    df['machine_wage'] = (df.bkt_rate_per_minutes * df.minutes).where(df.tool=='hummer',
                                                                      (df.hm_rate_per_minutes * df.minutes))
    df['machine_wage'] = df['machine_wage'].round()
    return df
 
def produce_machine_report(machine_staats_df):
    df = machine_staats_df[['date', 'company', 'brand', 'plate', 'driver_name','tool',
             'fuel', 'minutes', 'machine_wage', 'price_per_hour']]
    df = pd.pivot_table(df, aggfunc=np.sum, index=['company', 'brand', 
                                                   'plate', 'driver_name','tool',
                                                   'price_per_hour'])
    #estimate hours and minutes
    df['hours'] = df.minutes // 60
    df['minute'] = df.minutes % 60
    df['average_fuel'] = df.fuel / df.hours
    df2 = df.drop(columns=['minutes'])
 
    #rearrange columns to make sense
    df2 = df2[['hours', 'minute', 'fuel', 'average_fuel', 'machine_wage']]
    df2.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df2
 
def calculate_fuel(equipment_log, fuel):
    used_fuel = equipment_log.fuel.sum()
    supplied_fuel = fuel_supply.quantities.sum()
    remaining_fuel = supplied_fuel - used_fuel
 
    fuel = {'Supplied Fuel': supplied_fuel,
            'Used Fuel': used_fuel,
            'Fuel in Store':remaining_fuel}
 
    series = pd.Series(fuel)
    series.name = 'Fuel'
    df = series.to_frame()
    return df
 
def calculate_payroll(workers_april, workers_list):
    presence = workers_april.sum()
    presence.name = 'april_days'
    presence = presence.to_frame()
 
    workers_list['last_first_name'] = workers_list.last_name.apply(make_str_python_val) +\
                            '_'  + workers_list.first_name.apply(make_str_python_val)
    workers_list.set_index('last_first_name', inplace=True)
    salary_count = pd.merge(presence, workers_list, left_index=True, right_index=True)
    salary_count['total_pay'] = salary_count.april_days * salary_count.salary
    salary_count = salary_count[['last_name', 'first_name', 'role', 'april_days', 'salary', 'total_pay']]
    return salary_count

In [8]:
#unpack sheets from excel
equipment_db, equipment_log, fuel_supply, workers_list, fuel_consumption, workers_april, staff_list = dfs_from_xls(input_workbook)
sheets_from_xls = [equipment_db, equipment_log, fuel_supply, workers_list, fuel_consumption, workers_april, staff_list]

#calculate dataframes
machine_report = produce_machine_report(machine_staats(equipment_log, equipment_db))
fuel = calculate_fuel(equipment_log, fuel_supply)
payroll = calculate_payroll(workers_april, workers_list)

#total project cost dataframe
labour_cost = payroll.total_pay.sum()
fuel_cost = fuel.Fuel[0] * fuel_price
machine_wages_cost = machine_report.machine_wage.sum()
cost_series = pd.Series(data={'Labour Cost':labour_cost,
                              'Fuel Cost':fuel_cost,
                              'Machine Rent Cost':machine_wages_cost})
cost_series.name = 'Aproximate Cost'
cost_df = cost_series.to_frame()

#dataframes
df_list = [machine_report, fuel, payroll, cost_df]
df_names = ['machine_report', 'fuel', 'payroll', 'project_cost']

df_to_xls = dict(zip(df_names, df_list))

['equipment_db', 'equipment_log', 'fuel_supply', 'workers_list', 'fuel_consumption', 'workers_april', 'staff_list']


In [9]:
#display dataframes
from IPython.display import display
for df in df_list:
    display(df)

hours  ...  machine_wage
company        brand       plate   driver_name        tool    price_per_hour         ...              
ArtSEC         HOWO        RAD883B UWAMAHORO Joseph   Dumping 0                   0  ...           0.0
                           RAD884B MUNYANEZA Alphonse Dumping 0                   0  ...           0.0
BIB            Caterpillar RAC820Y Mapenzi            Bucket  50000               1  ...      120000.0
                                                      Hammer  100000             22  ...     2291667.0
CHIRWA Company JCB         RAD419P Yves               Hammer  100000             50  ...     5013334.0
               SUNNY       RAE003N Didier             Hammer  100000             35  ...     3503333.0
ECOGL          CMG         RAE826B AIME               Bucket  50000              41  ...     4594334.0
                                                      Hammer  110000             81  ...     8932000.0

[8 rows x 5 columns]

,Fuel
Supplied Fuel,6100.0
Used Fuel,4469.4
Fuel in Store,1630.6


,last_name,first_name,role,april_days,salary,total_pay
ufitimana_epaphrodite,UFITIMANA,Epaphrodite,Store Keeper,19,5000,95000
uwiringiyimana_jean_de_dieu,UWIRINGIYIMANA,Jean De Dieu,TT,18,2000,36000
twagiramungu_vedaste,TWAGIRAMUNGU,Vedaste,TT,18,2000,36000
uwiringiyimana_jean_de_dieu_guard,UWIRINGIYIMANA,Jean de Dieu Guard,Guard,19,1000,19000
twagiramungu_vedaste_guard,TWAGIRAMUNGU,Vedaste Guard,NaN,18,1000,18000


,Aproximate Cost
Labour Cost,204000.0
Fuel Cost,6405000.0
Machine Rent Cost,24454668.0


In [10]:
#style df to excel with
import openpyxl
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.styles import Alignment, Font, NamedStyle
from openpyxl.styles import Side, Border, numbers, PatternFill
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.worksheet.properties import WorksheetProperties, PageSetupProperties
from openpyxl.worksheet.cell_range import CellRange
 
#named styles
title_font_size = 14
top_row_font_size = 12
text_size = 11
 
 
##header style
title_style = NamedStyle(name="title_style")
title_style.font = Font(bold=True, size=title_font_size)
bd = Side(style='thin', color="000000")
title_style.border = Border(left=bd, top=bd, right=bd, bottom=bd)
title_style.alignment = Alignment(horizontal="left", vertical="center")
title_style.fill = PatternFill("solid", fgColor="EAFAF1")
 
##top style
top_row_style = NamedStyle(name="top_row_style")
top_row_style.font = Font(bold=True, size=top_row_font_size)
bd = Side(style='thin', color="000000")
top_row_style.border = Border(left=bd, top=bd, right=bd, bottom=bd)
top_row_style.alignment = Alignment(horizontal="center", vertical="center")
top_row_style.fill = PatternFill("solid", fgColor="D6EAF8")
 
##text style
text_style = NamedStyle(name="text_style")
text_style.font = Font(bold=False, size=text_size)
bd = Side(style='thin', color="000000")
text_style.border = Border(left=bd, top=bd, right=bd, bottom=bd)
text_style.alignment = Alignment(horizontal="left", vertical="center")
 
##number styles
numbers_style = NamedStyle(name="numbers_style")
numbers_style.font = Font(bold=False, size=text_size)
bd = Side(style='thin', color="000000")
numbers_style.border = Border(left=bd, top=bd, right=bd, bottom=bd)
numbers_style.alignment = Alignment(horizontal="right", vertical="center")
 
##SUM styles
sum_style = NamedStyle(name="sum_style")
sum_style.font = Font(bold=True, size=text_size, italic=True)
bd = Side(style='thin', color="000000")
sum_style.border = Border(left=bd, top=bd, right=bd, bottom=bd)
sum_style.alignment = Alignment(horizontal="right", vertical="center")
sum_style.fill = PatternFill("solid", fgColor="D6EAF8")
 
 
def insert_header(ws):
    merged_cells_range = ws.merged_cells.ranges
    for merged_cell in merged_cells_range:
        merged_cell.shift(0, 1)
    ws.insert_rows(1, 1)
    return ws
 
#size columns width
def as_text(value):
    if value is None:
        return ""
    return str(value)
 
def size_cols(ws):
    for column_cells in ws.columns:
        length = max(len(as_text(cell.value)) for cell in column_cells)
        ws.column_dimensions[column_cells[0].column].width = length + 3

def style_range(ws, range, style, top_row=False, numbers_style=False):
    for row in ws.iter_rows(min_row=range[0],
                            max_row=range[1],
                            min_col=range[2],
                            max_col=range[3]):
        for cell in row:
            if top_row and cell.value:
                cell.value = title_string(cell.value)
                cell.style = style
            elif numbers_style:
                cell.style = style
                cell.number_format = numbers.FORMAT_NUMBER_COMMA_SEPARATED1
            else:
                cell.style = style

def sum_column(ws, column, shift=1):
    column_sum = 0
    for row in ws.iter_rows(min_row=3,
                            max_row=ws.max_row,
                            min_col=column,
                            max_col=column):
        for cell in row:
            if cell.value:
                column_sum += cell.value
    sum_cell = ws.cell(ws.max_row+shift, column)
    sum_cell.value = column_sum

def sum_columns(ws, columns):
    if len(columns) > 1:
        sum_column(ws, columns[0])
        for column in columns[1:]:
            sum_column(ws, column, shift=0)
    else:
        sum_column(ws, columns[0])

def style_ws(ws, title_text, text_range, number_range,
             title_style, numbers_style, top_row_style, text_style,
             sum_style=None, columns_to_sum=None, total_text=None):
 
    #printing settings
    ws.page_setup.orientation = ws.ORIENTATION_LANDSCAPE
    ws.page_setup.paperSize = ws.PAPERSIZE_A4 
    # Property settings
    wsprops = ws.sheet_properties
    wsprops.tabColor = "1072BA"
    wsprops.filterMode = False
    wsprops.pageSetUpPr = PageSetupProperties(fitToPage=True, autoPageBreaks=False)
    wsprops.outlinePr.summaryBelow = False
    wsprops.outlinePr.applyStyles = True
 
    #shift all of the cells to make room for the title
    size_cols(ws)
    insert_header(ws)
    ##title
    title = ws['A1']
    title.value = title_text
    ws.merge_cells(start_column=ws.min_column,
                   end_column=ws.max_column,
                   start_row=ws.min_row,
                   end_row=ws.min_row)
    title.style = title_style
    
    ##top row
    style_range(ws=ws, range=[2, 2, 1, ws.max_column],
                style=top_row_style, top_row=True)
    
    ##text
    style_range(ws=ws, range=text_range, style=text_style)
    #numeric
    style_range(ws=ws, range=number_range, style=numbers_style, numbers_style=True)
    if columns_to_sum:
        #style sum text
        sum_columns(ws, columns_to_sum)
        style_range(ws=ws, range=[ws.max_row, ws.max_row, text_range[2],
                                  text_range[3]], style=sum_style)
        #merge sum text title cells
        ws.cell(ws.max_row, ws.min_column).value = total_text
        ws.merge_cells(start_column=ws.min_column,
                    end_column=text_range[3],
                    start_row=ws.max_row,
                    end_row=ws.max_row)
        #style sum numbers
        style_range(ws=ws, range=[ws.max_row, ws.max_row, number_range[2],
                                  number_range[3]], style=sum_style, numbers_style=True)
        

In [12]:
#write parsed reports to excel sheets
#and style sheets with openpyxl
with pd.ExcelWriter(output_workbook, engine='openpyxl') as writer:
    for sheet_name, df in df_to_xls.items():
        df.to_excel(writer, sheet_name=sheet_name)
    
    #Worksheets
    ws1 = writer.sheets['machine_report']
    ws2 = writer.sheets['fuel']
    ws3 = writer.sheets['payroll']
    ws3.delete_cols(1,1)
    ws4 = writer.sheets['project_cost']
    
    #sheets titles
    ws1_title = f"""
    ArtSEC
    Sewer Construction in Rubavu
    Summary of Machine Hours and Fuel
    {str(yesterday)}
    """
    
    ws2_title = f"""
        ArtSEC
        Sewer Construction
        Site Fuel Summary
        {str(yesterday)}
        """
    ws3_title = f"""
        ArtSEC
        Sewer Construction in Rubavu
        Summary of Payroll
        {str(yesterday)}
        """
    ws4_title = f"""
        ArtSEC
        Sewer Construction
        Total Project Cost
        {str(yesterday)}
        """
    #Style Sheets ws1
    style_ws(ws=ws1,
             title_text=ws1_title,
             text_range=[3, ws1.max_row+1, 1, 5],
             number_range=[3, ws1.max_row+1, 6, ws1.max_column],
             
             top_row_style=top_row_style,
             title_style=title_style,
             text_style=text_style,
             numbers_style=numbers_style,
             sum_style=sum_style,
             
             columns_to_sum=[9,11],
             total_text='Total Machine hours and renting cost')
    #Style Sheets ws2
    style_ws(ws=ws2,
             title_text=ws2_title,
             text_range=[3, ws2.max_row+1, 1, 1],
             number_range=[3, ws2.max_row+1, 2, ws2.max_column],
             
             top_row_style=top_row_style,
             title_style=title_style,
             text_style=text_style,
             numbers_style=numbers_style)
    
    #Style Sheets ws3
    style_ws(ws=ws3,
             title_text=ws3_title,
             text_range=[3, ws3.max_row+1, 1, 4],
             number_range=[3, ws3.max_row+1, 5, ws3.max_column],
             
             top_row_style=top_row_style,
             title_style=title_style,
             text_style=text_style,
             numbers_style=numbers_style,
             sum_style=sum_style,
             
             columns_to_sum=[ws3.max_column],
             total_text='April Payroll')
    
    #Style Sheets ws4
    style_ws(ws=ws4,
             title_text=ws4_title,
             text_range=[3, ws4.max_row+1, 1, 1],
             number_range=[3, ws4.max_row+1, 2, ws4.max_column],
             
             top_row_style=top_row_style,
             title_style=title_style,
             text_style=text_style,
             numbers_style=numbers_style,
             sum_style=sum_style,
             
             columns_to_sum=[ws4.max_column],
             total_text='Estimated Project Cost')
    print(f'Successfully saved to {output_workbook}')

Successfully saved to /content/drive/MyDrive/Pro/ArtSEC/Rock Excavation in Rubavu/site_report.xlsx
